In [2]:
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd

In [3]:
# 4131 page까지임 

#for helper in range(1, 43): #100page 단위 -> 잘 끊김
#for helper in range(1, 85): #50page 단위로
for helper in range(1, 3):  ###test
    driver = webdriver.Chrome("./chromedriver")
    
    ids = []
    titles = []
    paths = []
    contents = []
    dates = []
    nums = []
    url = ""

    i = (50 * (helper - 1)) + 1
    for page in range(i, 50+i):
    #for page in range(i, 1+i): ###test
        if page == 4132: #page는 4131 까지밖에 없음.
            break
        url = 'https://www1.president.go.kr/petitions/?c=42&only=2&page={}&order=1'.format(page)
        driver.get(url)
        print("current page : ", page)

        # driver.get(url)로 서버에게 get 요청을 보냈으니 response를 기다려야함. html을 받는데 걸리는 최소시간 -> 1.5s
        time.sleep(1.5)
        soup = bs(driver.page_source, 'html.parser') # 3초가 끝나면 새로운 page의 url로 접속하고 파서가 파싱한다.

        # 현 page에서 petition 요소에 접근
        lists = soup.find("div", attrs={"class": "ct_list1"})
        ul = lists.find("ul", attrs={"class": "petition_list"})
        a_lists = ul.find_all("a", attrs={"class": "cb relpy_w"})  # 현재 페이지의 "전체목록" 내부에 있는 모든 a 태그들을 리스트로 반환.
        id_lists = ul.find_all("div", attrs={"class": "bl_no"})
    
        # path를 추가하기 전에 현재 paths 리스트에 몇개가 들어있는지 확인한다 => 2 page가 되면 paths 리스트의 7번 index부터 읽어야하기 때문
        path_len = len(paths)
        
        for id in id_lists:
            #print(id.text[3:].strip()) id
            ids.append(id.text[3:].strip())
        
        for a in a_lists: # 현재 page 내에서 각 petition의 title과 path를 파싱하여 paths, titles 배열에 저장.
            # print(a.attrs["href"]) path 배열에 들어갈 경로 "/petition/12345"
            paths.append(a.attrs["href"])  # 각 청원의 path를 저장.
            # print(a.text[3:].strip()) titles
            titles.append(a.text[3:].strip())  # 각 청원의 title을 titles 배열에 저장.

        new_paths = paths[path_len:] # page 별로 해당하는 path만 읽어야하므로 => 2 page가 되면 paths 리스트의 7번 index부터 읽어야하기 때문
        print("new_paths : ", new_paths)
        for path in new_paths: # 현재 page의 모든 petition들의 해당 url에 접속하여 content를 파싱.
            url = "https://www1.president.go.kr" + path
            driver.get(url)
            soup = bs(driver.page_source, 'html.parser') # 각 url에 대하여 soup 객체를 새로 생성한다.

            # contents 추가하기.
            content = soup.find("div", attrs={"class":"View_write"})
            data = content.text.strip()
            data = data.replace("\n", "") # \n 문자도 제거
            #print('본문 내용 : ', data)

            # date 추가하기.
            date_ul = soup.find("ul", attrs={"class": "petitionsView_info_list"})
            li = date_ul.li.next_sibling.next_sibling
            li_data = li.text[4:]

            # num 추가하기.
            h2 = soup.find("h2", attrs={"class": "petitionsView_count"})
            num = h2.find('span', attrs={"class": "counter"})
            num = num.text

            contents.append(data) # 공백 제거한 data를 리스트에 저장.
            dates.append(li_data)
            nums.append(num)
    driver.close()
    # 최종 값
    print("id : ", ids)
    print("titles : ", titles)
    print("nums 배열의 수로 확인하기 ", len(nums)) # num의 갯수가 700개(100페이지 * 7개 청원글)여야 안뺴놓고 크롤링 한거임.
    #print("contents : ", contents)
    print("dates : ", dates)
    print("nums : ", nums)

    my_dictionary = {"id": ids, "title": titles, "date": dates, "num": nums, "contents": contents}
    data = pd.DataFrame(my_dictionary)  # 전체 데이터를 긁은 list인 total을 dataframe으로 변환시켜주면서 각 column의 이름을 부여해줍니다.

    str_i = str(i)
    str_i2 = str(i+49)
    datatitle = str_i + '_' + str_i2
    # datatitle = input("Please set the title of this excel file : ")  # 긁어온 데이터를 저장할 xlsx 파일의 이름을 지정해줍니다.
    data.to_excel("crawling_data/" + datatitle + ".xlsx", engine="xlsxwriter")
    #xlsxwriter를 임포트시켜서 engine을 추가하지 않으면, data.to_excel에서 IllegalCharacterError 에러가 뜬다.

current page :  1
new_paths :  ['/petitions/596422', '/petitions/596420', '/petitions/596415', '/petitions/596402', '/petitions/596365', '/petitions/596362', '/petitions/596354']
current page :  2
new_paths :  ['/petitions/596339', '/petitions/596328', '/petitions/596327', '/petitions/596325', '/petitions/596284', '/petitions/596268', '/petitions/596267']
current page :  3
new_paths :  ['/petitions/596248', '/petitions/596239', '/petitions/596220', '/petitions/596215', '/petitions/596210', '/petitions/596186', '/petitions/596181']
current page :  4
new_paths :  ['/petitions/596160', '/petitions/596153', '/petitions/596139', '/petitions/596108', '/petitions/596101', '/petitions/596092', '/petitions/596087']
current page :  5
new_paths :  ['/petitions/596083', '/petitions/596077', '/petitions/596072', '/petitions/596063', '/petitions/596052', '/petitions/596046', '/petitions/596042']
current page :  6
new_paths :  ['/petitions/596039', '/petitions/596010', '/petitions/596001', '/petition